## Week 2: Re-do initial classification on data 
Shateesh Bhugwansing

This notebook attempts to build a pipeline to use the Naive Bayes classifer on Subject 1, runs 1-4. 

### attempt 1: 
cv: 10-fold CV  
classifier pipeline: vectorizer --> MinMaxScaler --> BernoulliNB  
metric: accuracy

In [56]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
%matplotlib inline 

from sklearn.model_selection import StratifiedKFold
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn import metrics
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler


import warnings
warnings.simplefilter("ignore", DeprecationWarning)
plt.rcParams['figure.figsize'] = (8.0, 8.0)
plt.rcParams.update({'font.size': 12})


#import mne to read EEG data 
import mne 
from mne.decoding import Vectorizer


In [78]:
#read in the data for the first 4 runs of subject 1 
path1 = "/Users/shateeshbhugwansing/Desktop/seniorprojecteeg/clean_data_sample/Ex10_Suj1_Run1-epo.fif"
path2 = "/Users/shateeshbhugwansing/Desktop/seniorprojecteeg/clean_data_sample/Ex10_Suj1_Run2-epo.fif"
path3 = "/Users/shateeshbhugwansing/Desktop/seniorprojecteeg/clean_data_sample/Ex10_Suj1_Run3-epo.fif"
path4 = "/Users/shateeshbhugwansing/Desktop/seniorprojecteeg/clean_data_sample/Ex10_Suj1_Run4-epo.fif"
path_all = "/Users/shateeshbhugwansing/Desktop/seniorprojecteeg/clean_data_sample/subject1_all_runs-epo.fif"

data1 = mne.read_epochs(path1, preload=True)
data2 = mne.read_epochs(path2, preload=True)
data3 = mne.read_epochs(path3, preload=True)
data4 = mne.read_epochs(path4, preload=True)
data_all = mne.read_epochs(path_all, preload=True)

# extract only the event_id's 4-5 for new vs. scrambled 
data1 = data1[(data1.events[:,-1] == 4) | (data1.events[:,-1] == 5)]
data2 = data2[(data2.events[:,-1] == 4) | (data2.events[:,-1] == 5)]
data3 = data3[(data3.events[:,-1] == 4) | (data3.events[:,-1] == 5)]
data4 = data4[(data4.events[:,-1] == 4) | (data4.events[:,-1] == 5)]
data_all = data_all[(data_all.events[:,-1] == 4) | (data_all.events[:,-1] == 5)]


Reading /Users/shateeshbhugwansing/Desktop/seniorprojecteeg/clean_data_sample/Ex10_Suj1_Run1-epo.fif ...
Isotrak not found
    Read a total of 1 projection items:
        Average EEG reference (1 x 61) active
    Found the data of interest:
        t =       0.00 ...    1400.00 ms
        0 CTF compensation matrices available
280 matching events found
Created an SSP operator (subspace dimension = 1)
280 matching events found
Created an SSP operator (subspace dimension = 1)
1 projection items activated
Reading /Users/shateeshbhugwansing/Desktop/seniorprojecteeg/clean_data_sample/Ex10_Suj1_Run2-epo.fif ...
Isotrak not found
    Read a total of 1 projection items:
        Average EEG reference (1 x 61) active
    Found the data of interest:
        t =       0.00 ...    1400.00 ms
        0 CTF compensation matrices available
285 matching events found
Created an SSP operator (subspace dimension = 1)
285 matching events found
Created an SSP operator (subspace dimension = 1)
1 projection it

### attempt 1: data1

In [75]:
# make pipeline 
clf = make_pipeline(Vectorizer(),
                   MinMaxScaler(),
                   GaussianNB())

#cross-validation
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

labels1 = data1.events[:,-1]
preds1 = np.empty(len(labels1))

#fit the classifier + make predictions
for train, test in cv.split(data1, labels1):
    clf.fit(data1[train], labels1[train])
    preds1[test] = clf.predict(data1[test])


print("Attempt1, data1. accuracy (%): ", metrics.accuracy_score(labels1, preds1))


('Attempt1, data1. accuracy (%): ', 0.5673758865248227)


### attempt 1: data_all

In [ ]:
# make pipeline 
clf = make_pipeline(Vectorizer(),
                   MinMaxScaler(),
                   GaussianNB())

#cross-validation
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

labels_all = data_all.events[:,-1]
preds_all = np.empty(len(labels_all))

#fit the classifier + make predictions
for train, test in cv.split(data_all, labels_all):
    clf.fit(data1[train], labels1[train])
    preds1[test] = clf.predict(data1[test])


print("Attempt1, data1. accuracy (%): ", metrics.accuracy_score(labels1, preds1))